<a href="https://colab.research.google.com/github/aysucengiz/ceng463-hw2/blob/main/CENG463_hw2_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2  
Given a parliamentary speech in one of several languages, identify whether the speaker’s
party is currently governing (0) or in opposition (1).

* Select only one country (excluding the UK) and use the parliamentary debates from that country to complete the assigned tasks.

* Fine-tune a **multilingual masked language model:**
  - multilingual BERT
  - XLMRoberta-base
  - language-specific models like Turkish BERT or German BERT.

* In addition, you are required to experiment with a multilingual causal language model:
  - Llama-3.1-8B

## Masked LM

Fine-tune the selected masked language model for each task: For one task use ”text_en” and
for the other task use ”text” (original language).

## Causal LM

For each task, perform inference using the selected causal language model twice: Once using
”text en” and once using ”text” (original language).